In [87]:
import os
import pandas as pd
csv_path = os.getenv("HOME") + "/workplace/aiffel/Explore/03. news_crawler/news_dataset/total_data.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news'] = df['news'].str.replace("\n","")
df['news']

0       기업은행 뉴욕지점의 년 이란과의 금융거래 혐의에 대해 이 같은 결정을 내렸다고 관련...
1       뉴욕뉴시스월일현지시간 미국 뉴욕 브루클린의 한 쉐이크쉑 매장에 빵이 배달된 모습  ...
2       기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...
3       기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...
4       미국 텍사스의 비축유 시설로이터연합뉴스 미국 텍사스의 비축유 시설로이터연합뉴스월월물...
                              ...                        
9855    미국 이탈리아 등 개국서 유튜브가 일 현재 제공 중인 코로나 전용 섹션 서비스 한국...
9856    코리아는 자사가 서비스하는 무협  뇌명천하에 새로운 직업 궁수를 업데이트했다고 일 ...
9857    연세대암병원 제공 연세대암병원 제공연세암병원은 이달 일 오전 서울 신촌의 병원 층 ...
9858    출처아이뉴스 출처아이뉴스아이뉴스 송혜리 기자 이동통신사 유통점의 전산 운영시간 단축...
9859    동영상 강의 녹화 중이달 일 오후 울산대학교 강의실에서 첨단소재공학부전은채 교수가 ...
Name: news, Length: 9860, dtype: object

In [82]:
df.drop_duplicates(subset=['news'], inplace=True)

# print('뉴스 기사의 개수: ',len(df))

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [84]:
from konlpy.tag import Mecab
tokenizer = Mecab()

# 불용어 제거
stopwords = ['로이터', '채널', '출처', '뉴시스', '및', '연합뉴스', '지디넷코리아', '노컷뉴스', '한국일보', '해럴드경제', '아이뉴스', '로고뉴스', '사진제공', '전자신문', '무단전재', '에','는','은','을', '로', '되','를','당','채널','됐','경우','게','으로','했','에게', '에서', '있','적', '인', '께','고','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','라고', '아이뉴스','뉴스레터', '전자신문인터넷','안내기사의', '앵커', '이데일리', '입니다', '중복', '무단전재', '재배포', '닫기', '도', '섹션'] 

#토큰화 및 토큰화 과정에서 불용어를 제거하는 함수
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [85]:
text_data = preprocessing(df['news'])
# print(text_data[8])

In [86]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

# 데이터 학습
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

# 테스트
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

# 예측
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.00      0.00      0.00        59
          경제       0.63      0.98      0.77       233
          사회       0.92      0.93      0.93       132
       생활/문화       0.95      0.34      0.50       111

    accuracy                           0.73       535
   macro avg       0.63      0.56      0.55       535
weighted avg       0.70      0.73      0.67       535



/home/aiffel0047/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
